Our aim is to prepare a model that will predict the severity of car accidents in the city of Seattle for a set of conditions. In other words, given the weather and the road conditions what is the possibility of a commuter getting into a car accident and how severe it would be. 
So lets analyse our data which is available at the IBM cloud storage -<br>
cloudpath = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv" <br>
The source of this dataset is SDOT Traffic collisions report provided by Seattle Police Department. This dataset includes all type of collisions which happened in SDOT Traffic Division from 2004 to present. The dataset consists of 37 features (or columns) and 194763 entries (or rows). To analyse the data we need to carry out Data Wrangling or cleaning. 


In [1]:
# import the necessary libraries
import pandas as pd 
import numpy as np 
print('Pandas version =', pd.__version__)
print('Numpy version =', np.__version__)

Pandas version = 1.0.1
Numpy version = 1.19.0


In [2]:
# Reading the csv file into pandas dataframe
dfC = pd.read_csv("Project_Data/Data-Collisions.csv")
print("csv file successfully read in dataframe")
dfC.head() # display the first 5 rows of dataframe

csv file successfully read in dataframe


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [3]:
# Seing the dataset, SeverityCode is our target or dependent variable
# the missing values and empty cells of the dataframe have been replaced with nan values
# let us check the Datatype and non-null count of each feature
dfC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEVERITYCODE    194673 non-null  int64  
 1   X               189339 non-null  float64
 2   Y               189339 non-null  float64
 3   OBJECTID        194673 non-null  int64  
 4   INCKEY          194673 non-null  int64  
 5   COLDETKEY       194673 non-null  int64  
 6   REPORTNO        194673 non-null  object 
 7   STATUS          194673 non-null  object 
 8   ADDRTYPE        192747 non-null  object 
 9   INTKEY          65070 non-null   float64
 10  LOCATION        191996 non-null  object 
 11  EXCEPTRSNCODE   84811 non-null   object 
 12  EXCEPTRSNDESC   5638 non-null    object 
 13  SEVERITYCODE.1  194673 non-null  int64  
 14  SEVERITYDESC    194673 non-null  object 
 15  COLLISIONTYPE   189769 non-null  object 
 16  PERSONCOUNT     194673 non-null  int64  
 17  PEDCOUNT  

In [4]:
# we can also count the nan or missing values of each feature
dfC.isnull().sum()

SEVERITYCODE           0
X                   5334
Y                   5334
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
EXCEPTRSNCODE     109862
EXCEPTRSNDESC     189035
SEVERITYCODE.1         0
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        6329
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SDOTCOLNUM         79737
SPEEDING          185340
ST_COLCODE            18
ST_COLDESC          4904
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

In [5]:
# list out all the names of the features (or columns)
dfC.columns.values

array(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY',
       'REPORTNO', 'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION',
       'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC',
       'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT',
       'VEHCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER',
       'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING',
       'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY',
       'HITPARKEDCAR'], dtype=object)

Now on analysing the dataset and its features we can drop the following columns from our dataframe <br>
1) feature 'SEVERITYCODE.1' - since its the duplicate feature <br>
2) features 'INTKEY', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'INATTENTIONIND', 'PEDROWNOTGRNT', 'SDOTCOLNUM' - Since most of the entries in these columns are empty which cannot be replaced with any other value <br>
3) feature 'SPEEDING' - although speeding causes accident but in our case 95% of the entries are nan and also it is not required as per our problem statement <br>
4) features 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS', 'ADDRTYPE', 'LOCATION', 'UNDERINFL', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR' - these additional features can be safely dropped as they are not reuired as per our problem statement <br>
5) features 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE' - these features can also be safely dropped


In [6]:
drop_cols = ['SEVERITYCODE.1', 'INTKEY', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'INATTENTIONIND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS', 'ADDRTYPE', 'LOCATION', 'UNDERINFL', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE']
dfC.drop(drop_cols, axis=1, inplace=True)
dfC.shape

(194673, 11)

In [7]:
dfC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   SEVERITYCODE   194673 non-null  int64 
 1   SEVERITYDESC   194673 non-null  object
 2   COLLISIONTYPE  189769 non-null  object
 3   VEHCOUNT       194673 non-null  int64 
 4   SDOT_COLCODE   194673 non-null  int64 
 5   SDOT_COLDESC   194673 non-null  object
 6   WEATHER        189592 non-null  object
 7   ROADCOND       189661 non-null  object
 8   LIGHTCOND      189503 non-null  object
 9   ST_COLCODE     194655 non-null  object
 10  ST_COLDESC     189769 non-null  object
dtypes: int64(3), object(8)
memory usage: 16.3+ MB


SO after the first initial cleaning process we have reduced the features from 37 to only 11 which we will be using for model prediction. <br>
The Final Dataset consists of the following features:- <br>
1) SEVERITYCODE -INT- Code that corresponds to the severity of the collision<br>
2) SEVERITYDESC -TEXT- Detailed description of the severity of the collision<br>
3) COLLISIONTYPE -TEXT- The type of Collision<br>
4) VEHCOUNT -INT- The number of vehicles involved in the collision<br>
5) SDOT_COLCODE -INT- A code given to the collision by SDOT<br>
6) SDOT_COLDESC -TEXT- A description of the collision corresponding to the collision code<br>
7) WEATHER -TEXT- Description of the weather conditions during the time of the collision<br>
8) ROADCOND -TEXT- The condition of the road during the collision<br>
9) LIGHTCOND -TEXT- The light conditions during the collision<br>
10) ST_COLCODE -TEXT- A code provided by the state that describes the collision<br>
11) ST_COLDESC -TEXT- A description that corresponds to the state’s coding designation<br>

On further analysis you will find that there are still some missing values in the above features. Additionally there are some 'unknown' and 'other' values in the dataset. There are two ways to handle these values - <br>
1) replace missing or unknown values with avg. value of the feature column but being categorical variable this is not feasible <br>
2) replace them with feature value with max. frequency but that might create bias <br>
So we will drop all these entries (rows) with missing and unknown values without affecting the dataset 
 

In [8]:
dfC.replace(['Unknown', 'Other'], np.nan, inplace=True)
dfC.dropna(axis=0, inplace=True)
dfC.reset_index(drop=True, inplace=True)
print(dfC.shape)
dfC.head()

(147462, 11)


,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,VEHCOUNT,SDOT_COLCODE,SDOT_COLDESC,WEATHER,ROADCOND,LIGHTCOND,ST_COLCODE,ST_COLDESC
0,2,Injury Collision,Angles,2,11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",Overcast,Wet,Daylight,10,Entering at angle
1,1,Property Damage Only Collision,Sideswipe,2,16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",Raining,Wet,Dark - Street Lights On,11,From same direction - both going straight - bo...
2,1,Property Damage Only Collision,Parked Car,3,14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",Overcast,Dry,Daylight,32,One parked--one moving
3,2,Injury Collision,Angles,2,11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",Raining,Wet,Daylight,10,Entering at angle
4,1,Property Damage Only Collision,Angles,2,11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",Clear,Dry,Daylight,10,Entering at angle


In [9]:
# Now lets save the cleaned data to our project folder. We will carryout the modelling work on this cleaned dataset.
dfC.to_csv("Project_Data/Cleaned_Data.csv")

In [10]:
dfC.SEVERITYCODE.value_counts()

1    97734
2    49728
Name: SEVERITYCODE, dtype: int64

As we all know we need to predict the Accident Severity, so SeverityCode is our target variable or dependent variable and all others are independent variable. This is an unbalance dataset as the frequency of different severity codes are not equal. We need to balance this dataset before doing the modelling. 